In [2]:
# importação
import pandas as pd
import numpy as np
from sklearn.svm import SVR

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error

## 1. Pegar dados das 3 ações bancárias ITSA4, BBS3, ABCB4 (Neste teste primeiro vamos levar em conta apenas ITSA4)
- Pegando os dados do Yahoo Finance vou iniciar os dados
- Cortei a base de dados dos ultimos 1000 dias
- Coloca como index a data, para usar como série temporal

In [32]:
originalDataset = pd.read_csv('original-data/ITSA4-20140919-20180919.csv')
#originalDataset['Date'] = pd.to_datetime(originalDataset['Date'])
#originalDataset = originalDataset.set_index(['Date'])
originalDataset.head()
len(originalDataset)

999

# 1.5 Faz o Windowing através do método
- Coloca a função de windowing
- Transforma o dataset em um np.array
- Usa a função de windowing no array
- Transforma o array em um dataset novamente

In [40]:
def prepareDatasetPredict5DaysAhead(dataset):
    # create a dataset with date, open, open-7 open-6
    values = pd.DataFrame(dataset.values)

    dateColumn = values[0] # Date column

    # Open Columns
    openColumn = values[1] # Column Open
    openOffset6Column = values[1].shift(6) # Open offset 6 column
    openOffset7Column = values[1].shift(7) # Open offset 7 column

    # High Columns
    highColumn = values[2] 
    highOffset6Column = values[2].shift(6) # High offset 6 column
    highOffset7Column = values[2].shift(7) # High offset 7 column

    # Low Columns
    lowColumn = values[3] # Column Open
    lowOffset6Column = values[3].shift(6)  # Low offset 6 column
    lowOffset7Column = values[3].shift(7) # Low offset 7 column

    # Close Columns
    closeColumn = values[4]
    closeOffset6Column = values[4].shift(6) # Close offset 6 column
    closeOffset7Column = values[4].shift(7) # Close offset 7 column


    dataframe = pd.concat([dateColumn, 
                           openColumn, openOffset6Column, openOffset7Column,
                           highColumn, highOffset6Column, highOffset7Column,
                           lowColumn, lowOffset6Column, lowOffset7Column,
                           closeColumn, closeOffset6Column, closeOffset7Column,
                          ], axis=1)

    dataframe.columns = ['date',
                         'open', 'open-6', 'open-7', 
                         'high', 'high-6', 'high-7', 
                         'low', 'low-6', 'low-7',
                         'close', 'close-6', 'close-7']
    dataframe = dataframe.dropna()
    return dataframe

def prepareDatasetPredict22DaysAhead(dataset):
    values = pd.DataFrame(dataset.values)

    dateColumn = values[0] # Date column

    # Open Columns
    openColumn = values[1] # Column Open
    openFirstOffsetColumn = values[1].shift(23) 
    openSecondOffsetColumn = values[1].shift(24) 

    # High Columns
    highColumn = values[2] 
    highFirstOffsetColumn = values[2].shift(23) 
    highSecondOffsetColumn = values[2].shift(24) 

    # Low Columns
    lowColumn = values[3] # Column Open
    lowFirstOffsetColumn = values[3].shift(23)  
    lowSecondOffsetColumn = values[3].shift(24) 

    # Close Columns
    closeColumn = values[4]
    closeFirstOffsetColumn = values[4].shift(23) 
    closeSecondOffsetColumn = values[4].shift(24) 


    dataframe = pd.concat([dateColumn, 
                           openColumn, openFirstOffsetColumn, openSecondOffsetColumn,
                           highColumn, highFirstOffsetColumn, highSecondOffsetColumn,
                           lowColumn, lowFirstOffsetColumn, lowSecondOffsetColumn,
                           closeColumn, closeFirstOffsetColumn, closeSecondOffsetColumn,
                          ], axis=1)

    dataframe.columns = ['date',
                         'open', 'open-23', 'open-24', 
                         'high', 'high-23', 'high-24', 
                         'low', 'low-23', 'low-24',
                         'close', 'close-23', 'close-24']
    dataframe = dataframe.dropna()
    return dataframe

def prepareDatasetPredict1DayAhead(dataset):
    values = pd.DataFrame(dataset.values)
    dateColumn = values[0] # Date column

    # Open Columns
    openColumn = values[1] # Column Open
    openFirstOffsetColumn = values[1].shift(2) 

    # High Columns
    highColumn = values[2] 
    highFirstOffsetColumn = values[2].shift(2) 

    # Low Columns
    lowColumn = values[3] # Column Open
    lowFirstOffsetColumn = values[3].shift(2)  

    # Close Columns
    closeColumn = values[4]
    closeFirstOffsetColumn = values[4].shift(2) 


    dataframe = pd.concat([dateColumn, 
                           openColumn, openFirstOffsetColumn,
                           highColumn, highFirstOffsetColumn,
                           lowColumn, lowFirstOffsetColumn, 
                           closeColumn, closeFirstOffsetColumn,
                          ], axis=1)

    dataframe.columns = ['date',
                         'open', 'open-2',
                         'high', 'high-2',
                         'low', 'low-2', 
                         'close', 'close-2']
    dataframe = dataframe.dropna()
    return dataframe

#print(prepareDatasetPredict1DayAhead(dataset=originalDataset))
#print(prepareDatasetPredict5DaysAhead(dataset=originalDataset))
#print(prepareDatasetPredict22DaysAhead(dataset=originalDataset))

           date     open   open-2     high   high-2      low    low-2  \
2    2014-09-23  7.49124   7.7891  7.62528  7.84867  7.38699  7.60294   
3    2014-09-24  7.40188  7.43911  7.52847  7.54336  7.34231  7.39443   
4    2014-09-25  7.42422  7.49124   7.4689  7.62528  7.27529  7.38699   
5    2014-09-26  7.29763  7.40188  7.61783  7.52847  7.26784  7.34231   
6    2014-09-29  6.98487  7.42422  7.23061   7.4689  6.96998  7.27529   
7    2014-09-30  6.99977  7.29763    7.037  7.61783  6.73169  7.26784   
8    2014-10-01  6.80615  6.98487  6.82849  7.23061   6.6051  6.96998   
9    2014-10-02  6.73169  6.99977  6.98487    7.037  6.67956  6.73169   
10   2014-10-03  6.91785  6.80615  7.05189  6.82849  6.86573   6.6051   
11   2014-10-06  7.80399  6.73169  7.95292  6.98487  7.43911  6.67956   
12   2014-10-07  7.60294  6.91785  7.66996  7.05189  7.43167  6.86573   
13   2014-10-08  7.62528  7.80399  7.62528  7.95292  7.23806  7.43911   
14   2014-10-09  7.52102  7.60294  7.59549  7.66996